# Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from plotly.subplots import make_subplots
from sympy import symbols, diff, lambdify

# Newton's Method Multivariate

In [ ]:
def newton_multi_var(grad_F, H, x_init):
    x_history = []  # List to store the history of point updates
    x = x_init
    while True:
        H_inv = np.linalg.inv(np.float64(H(x)))
        x_move = np.dot(H_inv, np.float64(grad_F(x)))
        x = x - x_move
        x_history.append(x.copy())  # Record the updated point
        if np.linalg.norm(x_move) < 1e-8:
            break

    return x, np.array(x_history)

# Define Function

In [ ]:
x_init = np.array([3, 2])
x, y = symbols("x y")
#f = x**3 + y**3 - 9 * x * y + 27
a = 1
b = 100
f = (a - x)**2 + b * (y - x**2)**2

In [ ]:
#Calculate Gradients
grad_x = f.diff(x)
grad_y = f.diff(y)
calculate_grad_f = lambda val: np.array(
    [
        grad_x.subs([(x, val[0]), (y, val[1])]),
        grad_y.subs([(x, val[0]), (y, val[1])]),
    ]
)

#Calculate Hessian
H = lambda val: np.array(
    [
        [f.diff(x, x).subs([(x, val[0]), (y, val[1])]),f.diff(x, y).subs([(x, val[0]), (y, val[1])])],
        [f.diff(x, y).subs([(x, val[0]), (y, val[1])]),f.diff(y, y).subs([(x, val[0]), (y, val[1])])]
    ]
)

# Apply Newtons Method

In [ ]:
local_min ,  history  = newton_multi_var(calculate_grad_f, H,x_init)
round_local_min_pos = np.array([round(local_min[0], 3), round(local_min[1], 3)])
print("Local min: ({0},{1})".format(round_local_min_pos[0], round_local_min_pos[1]))

In [ ]:
history

# Visaulise

In [ ]:
"""# Redefine the function not as lambda function
def f(x, y):
    return x**3 + y**3 - 9 * x * y + 27"""

In [ ]:
def f(x, y, a=1, b=100):
    return (a - x)**2 + b * (y - x**2)**2

In [ ]:
#f = lambda val: val[0] ** 3 + val[1] ** 3 - 9 * val[0] * val[1] + 27
#f = lambda x, y: (1 - x)**2 + 100 * (y - x**2)**2
x = np.linspace(-6, 6, 30)
y = np.linspace(-6, 6, 30)

In [ ]:
X, Y = np.meshgrid(x, y)
Z = f(X,Y)

In [ ]:
ax = plt.axes(projection="3d")
ax.set_title("y = x**3 + y**3 - 9xy + 27")
ax.contour3D(X, Y, Z, 30)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.scatter([local_min[0]], [local_min[1]], [f(local_min[0],local_min[1])], color="red")
label = "Local min: ({0},{1})".format(
    round_local_min_pos[0], round_local_min_pos[1]
)
#ax.text(round_local_min_pos[0], round_local_min_pos[1], f(local_min), label, None)
plt.show()

In [ ]:
# Initialize figure with subplots
fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'surface'}]])

# Create a 3D surface plot with contours
fig.add_trace(go.Surface(x=X, y=Y, z=Z, showscale=False,
                         contours=dict(z=dict(color="limegreen",
                                              show=True,
                                              highlight=True,
                                              project=dict(x=True, y=True,z=True),
                                              size=1,
                                              start=0, end=30))),row=1, col=1)


# Add scatter plot trace to the figure
scatter_data = go.Scatter3d(x=history[:, 0],
                            y=history[:, 1],
                            z=f(history[:, 0],
                                history[:, 1]),
                            mode='lines+markers', 
                            marker=dict(symbol='x',
                                        color='red'))

fig.add_trace(scatter_data, row=1, col=1)


# Define animation frames
frames = []
for i in range(len(history)):
    frame_data = [go.Surface(x=X, y=Y, z=Z, contours=dict(z=dict(color="limegreen",
                                                                 show=True,
                                                                 highlight=True,
                                                                 project=dict(x=True,
                                                                              y=True,
                                                                              z=True),
                                                                 size=50,
                                                                 start=0, 
                                                                 end=1000))),
                  
                  go.Scatter3d(x=history[:i, 0],
                               y=history[:i, 1],
                               z=f(history[:i, 0],
                                   history[:i, 1]),
                               mode='lines+markers', 
                               marker=dict(symbol='circle',
                                           color='white',
                                           size=5))]

    frame = go.Frame(data=frame_data, name=f'Frame {i}')
    frames.append(frame)

# Add frames to the animation
fig.frames = frames

# Create animation buttons
animation_buttons = [
    dict(label="Play",
         method="animate",
         args=[None, {"frame": {"duration": 500, "redraw": True}, "fromcurrent": True}]),
    dict(label="Pause",
         method="animate",
         args=[[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate",
                        "transition": {"duration": 0}}])
]

# Update layout with animation settings
fig.update_layout(updatemenus=[{"buttons": animation_buttons, "type": "buttons", "showactive": False}],
                  title="Newton's Method Animation")

fig.update_layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=dict(l=50, r=50, b=100, t=100, pad=4),
    scene=dict(
        xaxis_title="X Axis",  # Set X-axis title
        yaxis_title="Y Axis",  # Set Y-axis title
        zaxis_title="Z Axis",  # Set Z-axis title
    ),
    showlegend=False,
    template="plotly_dark"
)

# Show the figure
fig.show()

In [ ]:
fig.write_html("NewtonSurface.html")

In [ ]:
fig = go.Figure(data =
         go.Contour(
           z= Z,
           colorbar=dict(nticks=10, 
                         ticks='outside',
                         ticklen=5, 
                         tickwidth=1,
                         showticklabels=True,
                         tickangle=0, 
                         tickfont_size=12)
            ))

fig.add_trace(go.Scatter(x=history[:, 0], 
                         y=history[:, 1], 
                         mode='markers+lines', 
                         name='Newton Method',
                         marker=dict(color='white')))

# Update layout to increase size
fig.update_layout(
    width=1000,  # Set the width of the figure
    height=1000,  # Set the height of the figure
    title='Contour Plot',  # Add a title to the plot
    xaxis_title="X Axis",  # Label the x-axis
    yaxis_title="Y Axis",  # Label the y-axis
    template="plotly_dark"  # Use a dark theme for the plot
)


fig.show()

In [ ]:
fig.write_html("NewtonContour.html")